# Clustering Crypto

In [136]:
# Initial imports
import pandas as pd
import hvplot.pandas
from path import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


### Deliverable 1: Preprocessing the Data for PCA

In [137]:
# Load the crypto_data.csv dataset.
file = "Resources/crypto_data.csv"
crypto_df= pd.read_csv(file, index_col = 0)
crypto_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [138]:
# Keep all the cryptocurrencies that are being traded.
crypto_df = crypto_df.loc[crypto_df["IsTrading"] == True]
crypto_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1144 entries, 42 to XBC
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CoinName         1144 non-null   object 
 1   Algorithm        1144 non-null   object 
 2   IsTrading        1144 non-null   bool   
 3   ProofType        1144 non-null   object 
 4   TotalCoinsMined  685 non-null    float64
 5   TotalCoinSupply  1144 non-null   object 
dtypes: bool(1), float64(1), object(4)
memory usage: 54.7+ KB


In [139]:
# Keep all the cryptocurrencies that have a working algorithm
for i in pd.unique(crypto_df["Algorithm"].values.ravel()):
    print (i)
crypto_df = crypto_df.loc[crypto_df['Algorithm'] != 'N/A']
len(pd.unique(crypto_df["Algorithm"].values.ravel()))

Scrypt
X11
SHA-256
X13
Ethash
CryptoNight-V7
Equihash
SHA-512
Multiple
X15
NIST5
Quark
Groestl
PoS
NeoScrypt
SHA3
HybridScryptHash256
Scrypt-n
PHI1612
Lyra2REv2
CryptoNight
Shabal256
Counterparty
Blake
Momentum
Stanford Folding
QuBit
XG Hash
M7 POW
Curve25519
Lyra2RE
QUAIT
vDPOS
Blake2b
BLAKE256
1GB AES Pattern Search
Dagger
CryptoNight-Lite
X11GOST
SHA-256D
POS 3.0
Progressive-n
DPoS
Lyra2Z
X14
Time Travel
Argon2
Keccak
Blake2S
Dagger-Hashimoto
536
Argon2d
Cloverhash
Skein
SkunkHash v2 Raptor
VeChainThor Authority
Ouroboros
POS 2.0
SkunkHash
C11
Proof-of-BibleHash
SHA-256 + Hive
Proof-of-Authority
XEVAN
VBFT
YescryptR16
IMesh
Green Protocol
Semux BFT consensus
X16R
Tribus
CryptoNight Heavy
Jump Consistent Hash
HMQ1725
Zhash
Cryptonight-GPU
ECC 256K1
CryptoNight Heavy X
Rainforest
Exosis
CryptoNight-lite
QuarkTX
Leased POS
Equihash1927
T-Inside
SHA3-256
Equihash+Scrypt
Slatechain
TRC10


89

In [140]:
# Remove the "IsTrading" column. 
crypto_df = crypto_df.drop(columns = 'IsTrading', inplace = False)

In [141]:
# Remove rows that have at least 1 null value.
crypto_df.isnull().sum()
crypto_df = crypto_df.dropna()
crypto_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 685 entries, 42 to XBC
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CoinName         685 non-null    object 
 1   Algorithm        685 non-null    object 
 2   ProofType        685 non-null    object 
 3   TotalCoinsMined  685 non-null    float64
 4   TotalCoinSupply  685 non-null    object 
dtypes: float64(1), object(4)
memory usage: 32.1+ KB


In [142]:
# Keep the rows where coins are mined.
crypto_df.isnull().sum()
crypto_df=crypto_df.loc[crypto_df["TotalCoinsMined"]>0]
crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0


In [143]:
# Create a new DataFrame that holds only the cryptocurrencies names.
CoinName_df = pd.DataFrame(data = crypto_df, columns = ["CoinName"])
CoinName_df

,CoinName
42,42 Coin
404,404Coin
1337,EliteCoin
BTC,Bitcoin
ETH,Ethereum
...,...
ZEPH,ZEPHYR
GAP,Gapcoin
BDX,Beldex
ZEN,Horizen


In [144]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
crypto_df = crypto_df.drop(columns = "CoinName")

In [145]:
# Use get_dummies() to create variables for text features.
crypto_dummies= pd.get_dummies(crypto_df, columns=["Algorithm", "ProofType"])
crypto_dummies.head()

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
42,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [146]:
# Standardize the data with StandardScaler().
df_scaled = StandardScaler().fit_transform(crypto_dummies)
print(df_scaled[0:1])

[[-0.11710817 -0.1528703  -0.0433963  -0.0433963  -0.0433963  -0.06142951
  -0.07530656 -0.0433963  -0.06142951 -0.06142951 -0.0433963  -0.0433963
  -0.19245009 -0.06142951 -0.09740465 -0.0433963  -0.11547005 -0.07530656
  -0.0433963  -0.0433963  -0.15191091 -0.0433963  -0.13118084 -0.0433963
  -0.0433963  -0.08703883 -0.0433963  -0.0433963  -0.0433963  -0.0433963
  -0.06142951 -0.0433963  -0.08703883 -0.08703883 -0.08703883 -0.0433963
  -0.13118084 -0.13840913 -0.13840913 -0.0433963  -0.06142951 -0.0433963
  -0.07530656 -0.18168574 -0.0433963  -0.0433963  -0.0433963  -0.07530656
  -0.15826614 -0.31491833 -0.0433963  -0.08703883 -0.07530656 -0.06142951
   1.38675049 -0.0433963  -0.0433963  -0.06142951 -0.0433963  -0.0433963
  -0.0433963  -0.0433963  -0.0433963  -0.0433963  -0.0433963  -0.0433963
  -0.39879994 -0.0433963  -0.18168574 -0.0433963  -0.08703883 -0.08703883
  -0.10680283 -0.0433963  -0.13118084 -0.0433963  -0.0433963  -0.0433963
  -0.0433963  -0.07530656 -0.43911856 -0.04339

### Deliverable 2: Reducing Data Dimensions Using PCA

In [147]:
# Using PCA to reduce dimension to three principal components.
pca = PCA(n_components=3)
df_pca = pca.fit_transform(df_scaled)
df_pca

array([[-0.32329007,  1.02285227, -0.55773753],
       [-0.30655316,  1.02284452, -0.55837513],
       [ 2.32034891,  1.60725382, -0.64795794],
       ...,
       [ 0.31861631, -2.32894011,  0.41779991],
       [-0.17057397, -1.95245781,  0.33165099],
       [-0.26312812,  0.78840961, -0.24860484]])

In [148]:
# Create a DataFrame with the three principal components.
pcs_df = pd.DataFrame(
    data = df_pca, columns = ['PC1', 'PC2', 'PC3'], index = crypto_df.index)

pcs_df

,PC1,PC2,PC3
42,-0.323290,1.022852,-0.557738
404,-0.306553,1.022845,-0.558375
1337,2.320349,1.607254,-0.647958
BTC,-0.148663,-1.325551,0.217725
ETH,-0.160677,-2.023110,0.441111
...,...,...,...
ZEPH,2.417606,0.924955,-0.223598
GAP,-0.321326,1.022712,-0.557775
BDX,0.318616,-2.328940,0.417800
ZEN,-0.170574,-1.952458,0.331651


### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [149]:
# Create an elbow curve to find the best value for K.
inertia = []
k = list(range(1, 11))
# Calculate the inertia for the range of K values
for i in k:
   km = KMeans(n_clusters=i, random_state=0)
   km.fit(pcs_df)
   inertia.append(km.inertia_)

elbow_data = {"k":k,"inertia":inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k",y="inertia",xticks=k,title="Elbow Curve")

C:\Users\walte\anaconda3\envs\mlenv\lib\site-packages\sklearn\cluster\_kmeans.py:882: UserWarning:

KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.



:Curve   [k]   (inertia)

Running K-Means with `k=4`

In [150]:
# Initialize the K-Means model.
model = KMeans(n_clusters=4, random_state=0)

# Fit the model
model.fit(pcs_df)

# Predict clusters
predictions = model.fit_predict(pcs_df)

In [151]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
clustered_df = pd.concat([crypto_df,pcs_df],axis = 1)

#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
clustered_df["CoinName"] = CoinName_df["CoinName"]
clustered_df.head()
#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
clustered_df["Class"] = model.labels_
# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)


(532, 9)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC1,PC2,PC3,CoinName,Class
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.323290,1.022852,-0.557738,42 Coin,0
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.306553,1.022845,-0.558375,404Coin,0
1337,X13,PoW/PoS,2.927942e+10,314159265359,2.320349,1.607254,-0.647958,EliteCoin,0
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.148663,-1.325551,0.217725,Bitcoin,3
ETH,Ethash,PoW,1.076842e+08,0,-0.160677,-2.023110,0.441111,Ethereum,3
LTC,Scrypt,PoW,6.303924e+07,84000000,-0.159377,-1.129638,0.004360,Litecoin,3
DASH,X11,PoW/PoS,9.031294e+06,22000000,-0.405418,1.246527,-0.553725,Dash,0
XMR,CryptoNight-V7,PoW,1.720114e+07,0,-0.160089,-2.214460,0.422484,Monero,3
ETC,Ethash,PoW,1.133597e+08,210000000,-0.159112,-2.023230,0.441083,Ethereum Classic,3
ZEC,Equihash,PoW,7.383056e+06,21000000,-0.170573,-1.952458,0.331651,ZCash,3


### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [152]:
# Creating a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(clustered_df, x="PC1", y="PC2", z="PC3", color="Class", symbol="Class", width=800, 
                    hover_name="CoinName", hover_data=["Algorithm"])
fig.update_layout(legend=dict(x=0, y=1))
fig.show()


In [153]:
# Create a table with tradable cryptocurrencies.
tradable_crypto_table = clustered_df.hvplot.table(columns=['CoinName', 'Algorithm', 'ProofType', 'TotalCoinSupply', 
                                                  'TotalCoinsMined', 'Class'], sortable=True, selectable=True)
tradable_crypto_table 

:Table   [CoinName,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined,Class]

In [154]:
# Print the total number of tradable cryptocurrencies.
print(f'Tota Tradable cryptocurrencies are {len(clustered_df)}.')

Tota Tradable cryptocurrencies are 532.


In [155]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
X_cluster = clustered_df[['TotalCoinSupply', 'TotalCoinsMined']].copy()
X_cluster_scaled = MinMaxScaler().fit_transform(X_cluster)
X_cluster_scaled

array([[4.20000000e-11, 0.00000000e+00],
       [5.32000000e-04, 1.06585544e-03],
       [3.14159265e-01, 2.95755135e-02],
       ...,
       [1.40022261e-03, 9.90135079e-04],
       [2.10000000e-05, 7.37028150e-06],
       [1.00000000e-06, 1.29582282e-07]])

In [156]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
plot_df = pd.DataFrame(X_cluster_scaled, columns=['TotalCoinSupply', 'TotalCoinsMined'], index=clustered_df.index)

# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
plot_df['CoinName'] = clustered_df.CoinName


# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
plot_df['Class'] = clustered_df.Class

plot_df.head(10)

,TotalCoinSupply,TotalCoinsMined,CoinName,Class
42,4.200000e-11,0.000000,42 Coin,0
404,5.320000e-04,0.001066,404Coin,0
1337,3.141593e-01,0.029576,EliteCoin,0
BTC,2.100000e-05,0.000018,Bitcoin,3
ETH,0.000000e+00,0.000109,Ethereum,3
LTC,8.400000e-05,0.000064,Litecoin,3
DASH,2.200000e-05,0.000009,Dash,0
XMR,0.000000e+00,0.000017,Monero,3
ETC,2.100000e-04,0.000115,Ethereum Classic,3
ZEC,2.100000e-05,0.000007,ZCash,3


In [157]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
(plot_df.hvplot.scatter(x='TotalCoinsMined', y='TotalCoinSupply', by='Class',hover_cols='CoinName',height=300, width=500, legend = "top") +
plot_df.hvplot.table(["TotalCoinsMined","TotalCoinsMined","CoinName","Class"],height = 300, width=400))

:Layout
   .NdOverlay.I :NdOverlay   [Class]
      :Scatter   [TotalCoinsMined]   (TotalCoinSupply,CoinName)
   .Table.I     :Table   [TotalCoinsMined,TotalCoinsMined,CoinName,Class]